In [2]:
from fredapi import Fred
import pandas as pd

# Use your FRED API key
fred = Fred(api_key="306a3b46d80923127981cdabb3c9691f")

# Step 1: Pull Exports and Imports from FRED
exports = fred.get_series('EXPGSCA')  # Quarterly exports
imports = fred.get_series('IMPGSCA')  # Quarterly imports

# Step 2: Build net exports DataFrame
net_exports_df = pd.DataFrame({
    'Exports': exports,
    'Imports': imports
})
net_exports_df['Net_Exports'] = net_exports_df['Exports'] - net_exports_df['Imports']
net_exports_df['Net_Export_Change'] = net_exports_df['Net_Exports'].diff()
net_exports_df = net_exports_df.reset_index().rename(columns={'index': 'Date'})
net_exports_df['Date'] = pd.to_datetime(net_exports_df['Date'])

# Step 3: Load Nominal GDP from full path
gdp_df = pd.read_csv(r"C:/Users/troyr/OneDrive - Self Taught LLC/PythonDev/03-projects/fred-trade-analysis/data/GDPCA.csv")
gdp_df['Date'] = pd.to_datetime(gdp_df['Date'])
gdp_df.columns = ['Date', 'Nominal_GDP']

# Step 4: Merge GDP with net exports
merged = pd.merge(net_exports_df, gdp_df, on='Date', how='inner')
merged['Net_Export_Share'] = merged['Net_Exports'] / merged['Nominal_GDP'] * 100

# Step 5: Summary
print("=== Net Export Share Summary ===")
print(merged['Net_Export_Share'].describe())

print("\n=== Top Trade Surplus Periods ===")
print(merged.sort_values(by='Net_Export_Share', ascending=False).head(10))

print("\n=== Deepest Trade Deficits ===")
print(merged.sort_values(by='Net_Export_Share').head(10))


=== Net Export Share Summary ===
count    96.000000
mean     -1.348738
std       1.340683
min      -4.727548
25%      -1.943519
50%      -0.975724
75%      -0.438620
max       1.641114
Name: Net_Export_Share, dtype: float64

=== Top Trade Surplus Periods ===
         Date  Exports  Imports  Net_Exports  Net_Export_Change  Nominal_GDP  \
18 1947-01-01   88.041   52.189       35.852             13.604     2184.614   
17 1946-01-01   77.215   54.967       22.248             52.823     2209.911   
20 1949-01-01   68.701   58.744        9.957              1.511     2261.928   
19 1948-01-01   69.343   60.897        8.446            -27.406     2274.627   
51 1980-01-01  387.475  364.197       23.278             63.635     7257.316   
52 1981-01-01  392.197  373.708       18.489             -4.789     7441.485   
22 1951-01-01   73.717   72.160        1.557             10.833     2656.320   
46 1975-01-01  269.381  266.300        3.081             31.562     6060.875   
62 1991-01-01  708.47

In [3]:
# Step 1: Imports and Setup
import pandas as pd
from fredapi import Fred

# Initialize FRED with your API key
fred = Fred(api_key="306a3b46d80923127981cdabb3c9691f")

# Step 2: Pull Exports & Imports (Nominal, Quarterly)
exports = fred.get_series('EXPGSCA')
imports = fred.get_series('IMPGSCA')

# Step 3: Load Nominal GDP (GDPCA)
gdp_nominal = pd.read_csv(r"C:/Users/troyr/OneDrive - Self Taught LLC/PythonDev/03-projects/fred-trade-analysis/data/GDPCA.csv")
gdp_nominal.columns = ['Date', 'Nominal_GDP']
gdp_nominal['Date'] = pd.to_datetime(gdp_nominal['Date'])

# Step 4: Load Real GDP (GDPC1)
gdp_real = pd.read_csv(r"C:/Users/troyr/OneDrive - Self Taught LLC/PythonDev/03-projects/fred-trade-analysis/data/GDPC1.csv")
gdp_real.columns = ['Date', 'Real_GDP']
gdp_real['Date'] = pd.to_datetime(gdp_real['Date'])

# Step 5: Merge Nominal and Real GDP
gdp_merged = pd.merge(gdp_nominal, gdp_real, on='Date', how='inner')
gdp_merged['Divergence_%'] = ((gdp_merged['Nominal_GDP'] - gdp_merged['Real_GDP']) / gdp_merged['Real_GDP']) * 100

# Step 6: Prepare Net Exports DataFrame
net_exports = pd.DataFrame({
    'Date': exports.index,
    'Exports': exports.values,
    'Imports': imports.values
})
net_exports['Net_Exports'] = net_exports['Exports'] - net_exports['Imports']
net_exports['Net_Export_Change'] = net_exports['Net_Exports'].diff()
net_exports['Date'] = pd.to_datetime(net_exports['Date'])

# Step 7: Merge with GDP data
final = pd.merge(gdp_merged, net_exports, on='Date', how='inner')
final['Net_Export_Share'] = final['Net_Exports'] / final['Nominal_GDP'] * 100

# Step 8: Correlation between Net Export Share and GDP Divergence
correlation = final[['Net_Export_Share', 'Divergence_%']].corr()

# Display results
print("📌 Correlation between Net Export Share and GDP Divergence (%):")
print(correlation)


📌 Correlation between Net Export Share and GDP Divergence (%):
                  Net_Export_Share  Divergence_%
Net_Export_Share          1.000000      0.081808
Divergence_%              0.081808      1.000000


In [4]:
# Correlation analysis segmented by historical periods

# Step 1: Define historical periods
final['Year'] = final['Date'].dt.year
final['Period'] = pd.cut(
    final['Year'],
    bins=[1946, 1971, 1985, 2001, 2020, 2030],  # Extend to 2030 to capture recent data
    labels=[
        'Bretton Woods (1947–71)',
        'Inflation/Volcker (1971–85)',
        'Globalization Boom (1985–01)',
        'China-WTO Era (2001–20)',
        'COVID & Reshoring (2020–)'
    ]
)

# Step 2: Drop rows without a period label (e.g., 1946 or NaT rows)
final_clean = final.dropna(subset=['Period'])

# Step 3: Group and compute correlation matrix per period
period_grouped = final_clean.groupby('Period')[['Net_Export_Share', 'Divergence_%']].corr().unstack().iloc[:, 1]

# Step 4: Display results
print("📊 Correlation Between Net Export Share and GDP Divergence by Historical Period:\n")
print(period_grouped)


📊 Correlation Between Net Export Share and GDP Divergence by Historical Period:

Period
Bretton Woods (1947–71)        -0.075112
Inflation/Volcker (1971–85)    -0.621027
Globalization Boom (1985–01)    0.002529
China-WTO Era (2001–20)        -0.018915
COVID & Reshoring (2020–)       0.512359
Name: (Net_Export_Share, Divergence_%), dtype: float64


C:\Users\troyr\AppData\Local\Temp\ipykernel_28560\168247836.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  period_grouped = final_clean.groupby('Period')[['Net_Export_Share', 'Divergence_%']].corr().unstack().iloc[:, 1]
